In [1]:
import os
import boto3
import re
import copy
import time
from time import gmtime, strftime
from sagemaker import get_execution_role

role = get_execution_role()

region = boto3.Session().region_name

bucket='sagemaker-linkage' # Replace with your s3 bucket name
prefix = 'sagemaker/xgboost-mnist' # Used as part of the path in the bucket where you store data
bucket_path = 'https://s3-{}.amazonaws.com/{}'.format(region,bucket) # The URL to access the bucket

In [2]:
!pip install recordlinkage
import recordlinkage

    100% |████████████████████████████████| 942kB 20.1MB/s ta 0:00:01
    100% |████████████████████████████████| 286kB 25.4MB/s ta 0:00:01
    100% |████████████████████████████████| 143kB 42.2MB/s ta 0:00:01
  Running setup.py bdist_wheel for jellyfish ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/e8/fe/99/d8fa8f2ef7b82a625b0b77a84d319b0b50693659823c4effb4
Successfully built jellyfish
You are using pip version 10.0.1, however version 19.2.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import pandas as pd
import numpy as np

In [4]:
from recordlinkage.datasets import load_febrl4
#public datasets from recordlinkage libary

In [5]:
df_a, df_b = load_febrl4()
df_a

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-1070-org,michaela,neumann,8,stanley street,miami,winston hills,4223,nsw,19151111,5304218
rec-1016-org,courtney,painter,12,pinkerton circuit,bega flats,richlands,4560,vic,19161214,4066625
rec-4405-org,charles,green,38,salkauskas crescent,kela,dapto,4566,nsw,19480930,4365168
rec-1288-org,vanessa,parr,905,macquoid place,broadbridge manor,south grafton,2135,sa,19951119,9239102
rec-3585-org,mikayla,malloney,37,randwick road,avalind,hoppers crossing,4552,vic,19860208,7207688
rec-298-org,blake,howie,1,cutlack street,belmont park belted galloway stud,budgewoi,6017,vic,19250301,5180548
rec-1985-org,NaN,lund,109,caley crescent,allandale aged care facility,mill park,4053,nsw,19180902,7074690
rec-2404-org,blakeston,broadby,53,traeger street,valley of springs,north ward,3083,qld,19120907,4308555
rec-1473-org,NaN,leslie,925,carpenter close,NaN,canterbury,2340,vic,19950608,2438058


In [6]:
df_b

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-561-dup-0,elton,NaN,3,light setreet,pinehill,windermere,3212,vic,19651013,1551941
rec-2642-dup-0,mitchell,maxon,47,edkins street,lochaoair,north ryde,3355,nsw,19390212,8859999
rec-608-dup-0,NaN,white,72,lambrigg street,kelgoola,broadbeach waters,3159,vic,19620216,9731855
rec-3239-dup-0,elk i,menzies,1,lyster place,NaN,northwood,2585,vic,19980624,4970481
rec-2886-dup-0,NaN,garanggar,NaN,may maxwell crescent,springettst arcade,forest hill,2342,vic,19921016,1366884
rec-4285-dup-0,sophie,manson,14,elizabeth xrescent,manorhouse,gorokan,3465,vic,19510201,4818868
rec-929-dup-0,madison,morrison,38,peacock place,melim a,campbelltown,3021,wa,19481214,1967265
rec-4833-dup-0,jayed,millar,6,meredith c ircuit,NaN,thornbury,6014,vic,19340731,1314310
rec-717-dup-0,joshua,raward,26,yambina cerscent,thorndale,macgrdgor,4718,qld,NaN,9086686


In [1]:
indexer = recordlinkage.SortedNeighbourhoodIndex(
    left_on = 'surname', right_on = 'surname', window = 3)
links = indexer.index(df_a,df_b)

NameError: name 'recordlinkage' is not defined

In [8]:
print(len(links))

97699


In [9]:
# This cell can take some time to compute.
compare_cl = recordlinkage.Compare()

compare_cl.string('given_name', 'given_name', method = 'jarowinkler', threshold = 0.85, label = 'name')
compare_cl.string('surname', 'surname', method='jarowinkler', threshold=0.85, label='surname')
compare_cl.exact('date_of_birth', 'date_of_birth', label='date_of_birth')
compare_cl.exact('suburb', 'suburb', label='suburb')
compare_cl.exact('state', 'state', label='state')
compare_cl.string('address_1', 'address_1', threshold=0.85, label='address_1')
compare_cl.exact('soc_sec_id', 'soc_sec_id', label = 'ss_id')

features = compare_cl.compute(links, df_a, df_b)
features.head()

name  surname  date_of_birth  suburb  state  \
rec_id_1     rec_id_2                                                      
rec-1070-org rec-2820-dup-0   0.0      0.0              0       0      1   
             rec-4348-dup-0   0.0      0.0              0       0      0   
             rec-2276-dup-0   0.0      0.0              0       0      0   
             rec-3382-dup-0   0.0      0.0              0       0      0   
rec-2797-org rec-2820-dup-0   0.0      0.0              0       0      0   

                             address_1  ss_id  
rec_id_1     rec_id_2                          
rec-1070-org rec-2820-dup-0        0.0      0  
             rec-4348-dup-0        0.0      0  
             rec-2276-dup-0        0.0      0  
             rec-3382-dup-0        0.0      0  
rec-2797-org rec-2820-dup-0        0.0      0

In [10]:
features.sum(axis=1).value_counts().sort_index(ascending=False)

7.0     1514
6.0     1660
5.0      554
4.0       95
3.0      159
2.0    19703
1.0    69886
0.0     4128
dtype: int64

In [11]:
high_match = features[features.sum(axis=1) > 5]
high_match.head()
#all the pairs with more than 3 one's in the features, simple but not effective

,,name,surname,date_of_birth,suburb,state,address_1,ss_id
rec_id_1,rec_id_2,,,,,,,
rec-4312-org,rec-4312-dup-0,1.0,1.0,1,0,1,1.0,1
rec-4372-org,rec-4372-dup-0,1.0,1.0,1,1,1,1.0,1
rec-170-org,rec-170-dup-0,1.0,1.0,1,0,1,1.0,1
rec-1991-org,rec-1991-dup-0,1.0,1.0,1,1,1,1.0,1
rec-2389-org,rec-2389-dup-0,1.0,1.0,1,1,1,1.0,0


In [12]:
high_match_tuple = list(high_match.index.values)

In [13]:
true_links = pd.MultiIndex.from_tuples(high_match_tuple, names=('id1', 'id2'))

In [14]:
train_pairs = features[0:1000]
train_pairs.head()

name  surname  date_of_birth  suburb  state  \
rec_id_1     rec_id_2                                                      
rec-1070-org rec-2820-dup-0   0.0      0.0              0       0      1   
             rec-4348-dup-0   0.0      0.0              0       0      0   
             rec-2276-dup-0   0.0      0.0              0       0      0   
             rec-3382-dup-0   0.0      0.0              0       0      0   
rec-2797-org rec-2820-dup-0   0.0      0.0              0       0      0   

                             address_1  ss_id  
rec_id_1     rec_id_2                          
rec-1070-org rec-2820-dup-0        0.0      0  
             rec-4348-dup-0        0.0      0  
             rec-2276-dup-0        0.0      0  
             rec-3382-dup-0        0.0      0  
rec-2797-org rec-2820-dup-0        0.0      0

In [15]:
match_index = train_pairs.index & true_links
# match_index

In [16]:
logreg = recordlinkage.LogisticRegressionClassifier()

In [17]:
logreg.fit(train_pairs, match_index)
print ("Intercept: ", logreg.intercept)
print ("Coefficients: ", logreg.coefficients)

Intercept:  -4.107511768674469
Coefficients:  [ 2.18051384 -1.52550991  1.6158325   1.38981158  0.32347552  1.5439147
  1.6802414 ]


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [18]:
result_logreg = logreg.predict(features)

len(result_logreg)

3463

In [19]:
recordlinkage.confusion_matrix(true_links, result_logreg, len(features))

array([[ 3174,     0],
       [  289, 94236]])

In [20]:
recordlinkage.fscore(true_links, result_logreg)

0.956456230224499

In [21]:
intercept = -6
coefficients = [-1.0, 1.0, 3.0, 0.0, 0.0, 3.0, 3.0]

logreg = recordlinkage.LogisticRegressionClassifier(coefficients, intercept)

# predict without calling LogisticRegressionClassifier.fit
result_logreg_pretrained = logreg.predict(features)
print (len(result_logreg_pretrained))

2784


In [22]:
recordlinkage.confusion_matrix(true_links, result_logreg_pretrained, len(features))

array([[ 2498,   676],
       [  286, 94239]])

In [23]:
recordlinkage.fscore(true_links, result_logreg_pretrained)

0.8385364216179926

In [27]:
small_match = features[features.sum(axis=1) > 3]
tuple_list = list(small_match.index.values)
tuple_df = pd.DataFrame(tuple_list, columns=['a', 'b'])
# tuple_df
merged = pd.merge(df_a, tuple_df, left_index=True, right_on='a')
merged = pd.merge(merged, df_b, right_index=True, left_on='b')
merged = merged.drop(['a','b'], axis=1)
merged

,given_name_x,surname_x,street_number_x,address_1_x,address_2_x,suburb_x,postcode_x,state_x,date_of_birth_x,soc_sec_id_x,given_name_y,surname_y,street_number_y,address_1_y,address_2_y,suburb_y,postcode_y,state_y,date_of_birth_y,soc_sec_id_y
231,courtney,painter,12,pinkerton circuit,bega flats,richlands,4560,vic,19161214,4066625,courtney,painter,12,pinkerton circuit,NaN,richlands,4560,vci,19161214,4066625
235,charles,green,38,salkauskas crescent,kela,dapto,4566,nsw,19480930,4365168,charles,green,38,salkauskas crescent,klea,dapto,4566,nsw,19480930,4365168
298,vanessa,parr,905,macquoid place,broadbridge manor,south grafton,2135,sa,19951119,9239102,vanessa,parr,905,macquoid place,broadbridge manor,south qrafton,2135,sa,19951119,9239102
305,mikayla,malloney,37,randwick road,avalind,hoppers crossing,4552,vic,19860208,7207688,mikayla,malloney,37,randwick road,avalsnd,hoppers crossing,4552,vic,19860208,7207688
306,blake,howie,1,cutlack street,belmont park belted galloway stud,budgewoi,6017,vic,19250301,5180548,blake,howie,1,belmont park belted galloway stud,cutlack street,budgewoi,6071,vic,19250301,5180548
310,NaN,lund,109,caley crescent,allandale aged care facility,mill park,4053,nsw,19180902,7074690,NaN,lund,287,caley crescent,allandale agedc are facility,mill aprk,4053,nsw,19180902,7074690
313,blakeston,broadby,53,traeger street,valley of springs,north ward,3083,qld,19120907,4308555,blakeston,broadby,53,traeger street,valley of springs,north ward,3083,NaN,19120907,4308555
316,NaN,leslie,925,carpenter close,NaN,canterbury,2340,vic,19950608,2438058,NaN,leslie,952,carpenter close,NaN,cantebrry,2340,vic,19950608,2438058
320,edward,denholm,10,corin place,gold tyne,clayfield,4221,vic,19660306,7119771,edward,denholm,10,corin place,goldt yne,clayfield,4221,vic,19130629,7119771
321,charlie,alderson,266,hawkesbury crescent,deergarden caravn park,cooma,4128,vic,19440908,1256748,charlie,alderson,266,NaN,deergarden caravn park,cooma,4128,vic,19440908,1256748
